In [16]:
import src.config as config

In [17]:
import pandas as pd

current_date = pd.Timestamp('2023-02-28 09:00:00')

## Get pointers to the 2 feature groups we need to create our feature view

- One with the model predictions
- The other with the actual demands

In [18]:
from src.feature_store_api import get_feature_group

predictions_fg = get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS
)

actuals_fg = get_feature_group(
    name=config.FEATURE_GROUP_NAME,
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/12447
Connected. Call `.close()` to terminate connection gracefully.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/12447
Connected. Call `.close()` to terminate connection gracefully.


## Query to generate our feature view from these 2 feature groups

In [19]:
from datetime import timedelta

query = predictions_fg.select_all() \
    .join(actuals_fg.select_all(), on=['pickup_hour', 'pickup_location_id']) \
    .filter(predictions_fg.pickup_hour >= current_date - timedelta(days=30))

In [20]:
from src.feature_store_api import get_feature_store

feature_store = get_feature_store()

try:
    # create feature view as it does not exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_MONITORING,
        version=1,
        query=query
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
predictions_and_actuals_fv = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_MONITORING,
    version=1
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/12447
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.


In [21]:
predictions_and_actuals = predictions_and_actuals_fv.get_batch_data(
    # start_time=(fetch_data_from - timedelta(days=1)),
    # end_time=(fetch_data_to + timedelta(days=1))
)

2023-02-28 12:06:35,552 INFO: USE `taxi_demand_featurestore`
2023-02-28 12:06:36,096 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`pickup_location_id` `pickup_location_id`, `fg1`.`predicted_demand` `predicted_demand`, `fg1`.`pickup_hour` `pickup_hour`, `fg1`.`pickup_location_id` `join_pk_pickup_location_id`, `fg1`.`pickup_hour` `join_pk_pickup_hour`, `fg1`.`pickup_hour` `join_evt_pickup_hour`, `fg0`.`rides` `rides`, RANK() OVER (PARTITION BY `fg1`.`pickup_hour`, `fg1`.`pickup_location_id`, `fg1`.`pickup_hour` ORDER BY `fg0`.`pickup_hour` DESC) pit_rank_hopsworks
FROM `taxi_demand_featurestore`.`model_predictions_feature_group_1` `fg1`
INNER JOIN `taxi_demand_featurestore`.`time_series_hourly_feature_group_1` `fg0` ON `fg1`.`pickup_hour` = `fg0`.`pickup_hour` AND `fg1`.`pickup_location_id` = `fg0`.`pickup_location_id` AND `fg1`.`pickup_hour` >= `fg0`.`pickup_hour`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`pickup_location_id` `pickup_location_id`, `right_fg0`.`pred

In [22]:
predictions_and_actuals

,pickup_location_id,predicted_demand,pickup_hour,rides
0,1,0.0,2023-02-27 10:00:00,0
1,2,0.0,2023-02-27 10:00:00,0
2,3,0.0,2023-02-27 10:00:00,0
3,4,5.0,2023-02-27 10:00:00,3
4,5,0.0,2023-02-27 10:00:00,0
...,...,...,...,...
525,261,11.0,2023-02-28 09:00:00,15
526,262,132.0,2023-02-28 09:00:00,155
527,263,147.0,2023-02-28 09:00:00,144
528,264,48.0,2023-02-28 09:00:00,61


In [23]:
predictions_and_actuals.sort_values(by=['predicted_demand'], ascending=False).head(20)

,pickup_location_id,predicted_demand,pickup_hour,rides
500,236,429.0,2023-02-28 09:00:00,349
501,237,365.0,2023-02-28 09:00:00,370
235,236,319.0,2023-02-27 10:00:00,323
236,237,287.0,2023-02-27 10:00:00,307
450,186,247.0,2023-02-28 09:00:00,272
185,186,225.0,2023-02-27 10:00:00,218
137,138,209.0,2023-02-27 10:00:00,248
426,162,207.0,2023-02-28 09:00:00,207
160,161,204.0,2023-02-27 10:00:00,195
434,170,202.0,2023-02-28 09:00:00,203


In [ ]:
import plotly.express as px 

fig = px.line(
    x=ts_dates, y=ts_values,
    template='plotly_dark',
    markers=True, title=title
)